<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-Font_Fix

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the Font Fix conservative-to-primitive routine available in GRHayL.

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#introduction): **Introduction**
1. [Step 2](#Noble_Routines): **`Noble Routines`**
    1. [Step 2.a](#variables_needed_by_Noble): *Setting up the variables needed by `Noble`*
        1. [Step 2.a.i](#Noble_vars): Conversion to Noble variables
        1. [Step 2.a.ii](#Noble_B): Computing $B^{i}_{\rm Noble}$
        1. [Step 2.a.iii](#Noble_prims): Writing $\boldsymbol{P}_{\rm Noble}$ in terms of $\boldsymbol{P}_{\rm GRHayL}$
    1. [Step 2.b](#equatorial_symmetry): *Applying equatorial symmetry*
    1. [Step 2.c](#variable_setup): *Setting up the variables needed by `HARM`*
        1. [Step 2.c.ii](#variable_setup__prims): Primitives
        1. [Step 2.c.iii](#variable_setup__conservs): Conservatives
        1. [Step 2.c.iv](#variable_setup__lapse_and_psi): Lapse function and conformal factor
        1. [Step 2.c.v](#variable_setup__phys_metric): Physical spatial metric
        1. [Step 2.c.vi](#variable_setup__betadown_and_beta2): $\beta_{i}$ and $\beta^{2} \equiv \beta_{i}\beta^{i}$
        1. [Step 2.c.vii](#variable_setup__adm_4metric): The ADM 4-metric, $g_{\mu\nu}$, and its inverse, $g^{\mu\nu}$
        1. [Step 2.c.viii](#variable_setup__temp_conservs): Temporary storage for current values of the conservative variables
    1. [Step 2.d](#conserv_to_prim__driver): *Determining the primitives variables from the conservatives variables*
    1. [Step 2.e](#enforce_limits_on_primitives_and_recompute_conservs): *Enforcing physical limits on primitives and recomputing the conservatives variables*
    1. [Step 2.f](#updating_conservs_and_prims_gfs): *Updating conservative and primitive gridfunctions*
    1. [Step 2.g](#diagnostics_and_debugging_tools): *Diagnostics and debugging tools*
1. [Step 3](#harm_primitives_lowlevel): **`harm_primitives_lowlevel.C`**
1. [Step 7](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

The `font_fix` conservative-to-primitive (C2P) function serves as a failsafe in the event of the failure of the other C2P methods. Currently, `font_fix` only supports Hybrid EOS. This function isn't included in the `Hybrid_Multi_Method`. To use it, we simply call `font_fix` directly, and it is usually triggered only if the return value from the main C2P method(s) is(are) non-zero. While `font_fix` has the same argument list as the other C2P functions, it does not require initial primitive guesses. The only necessary primitive inputs are $B^i$.

For more information on the arguments for conservative-to-primitive routines, see the [tutorial](Tutorial-Hybrid_Multi_Method.ipynb) for `Hybrid_Multi_Method`.

<a id='font_fix'></a>

# Step 2: The `font_fix` Function \[Back to [top](#toc)\]
$$\label{font_fix}$$

If the conservative-to-primitive solver fails to converge, we provide the procedure suggested by [Font *et al.* (1998)](https://arxiv.org/abs/gr-qc/9811015). The algorithm can be summarized as the requirement that

$$
P=P_{\rm cold} = \kappa \rho^{\Gamma_{\rm cold}}_{b}\ ,
$$

and then recomputing the velocities $u_{i}$. We will describe this procedure in detail in [Step 4](#font_fix_gamma_law__c) below.

## <a id='compute_utconi'></a>

## Step 2.d: Compute $\tilde{u}^{i}$ \[Back to [top](#toc)\]
$$\label{compute_utconi}$$

Now we evaluate

$$
\tilde{u}^{i} = \gamma^{ij}u_{i} \implies
\boxed{
\left\{
\begin{align}
\tilde{u}^{x} &= \gamma^{xx}u_{x} + \gamma^{xy}u_{y} + \gamma^{xz}u_{z}\\
\tilde{u}^{y} &= \gamma^{yx}u_{x} + \gamma^{yy}u_{y} + \gamma^{yz}u_{z}\\
\tilde{u}^{z} &= \gamma^{zx}u_{x} + \gamma^{zy}u_{y} + \gamma^{zz}u_{z}
\end{align}
\right.
}
$$

<a id='returned_prims'></a>

## Step 3.c: Returned values from `Noble` Routines \[Back to [top](#toc)\]
$$\label{returned_prims}$$

Once the `Newton-Rapson` method converges (or fails), it returns to the `Noble` routines with an error code. We will combine these error codes with those from `Noble` at the end of this Step. As mentioned in the [Introduction](#introduction), all routines return the quantity $Z$, and the `Noble2D` routine also returns the squared 4-velocity magnitude $u^2$. The various `Noble1D` routines instead compute $u^2$ from $Z$. If
$$u^2>1$$
it is set to
$$1-2*10^{-16}$$
Then, we begin computing the primitives. The baryonic density is
$$
\rho_b = \frac{\rho^{\rm undens}}{W}
$$
where the Lorenz factor $W=\left(1-u^2\right)^{-1/2}$. To compute the velocities, we compute the `HARM` variable $\tilde{u}^i$ and then compute $v^i$ from it. The full expression for $\tilde{u}^i$ is
$$
\tilde{u}^i = \frac{W}{Z+B^2}\left(\tilde{Q}^i + \frac{Q_j B^j}{Z} B^i \right)
$$
This is, unfortunately, going to have several definitions of variables in a row. These intermediate quantities are either used inside the `Newton-Rapson` method or are used to compute *other* intermediate quantities which are used in the `Newton-Rapson` method. We define them all here so that they are as close to the $\tilde{u}^i$ equation as possible. $Q^i$ and $\tilde{Q}^i$ are given by
\begin{align}
\tilde{Q}^i &\equiv Q^i + n^i Q^\mu n_\mu \\
Q^\mu &= g^{\mu\nu}Q_{\mu\nu} \\
Q_0 &\equiv u^{\rm Noble} \\
Q_i &\equiv S_i^{\rm Noble} \\
n_\mu &= -\alpha \delta^0_\mu \\
n^\mu &= -\frac{1}{\alpha}\{1, \beta^i\}
\end{align}
where $n_\mu$ is the normal vector to the spatial hypersurface.
For completeness, we rework the $\tilde{u}^i$ equation to be in terms of the input variables
\begin{align}
\tilde{u}^i &= \frac{W}{Z+B^2}\left(\tilde{Q}^i + \frac{Q_j B^j}{Z} B^i \right) \\
%
&= \frac{W}{Z+B^2}\left(Q^i + n^i Q^j n_j + \frac{S_j^{\rm Noble} B^j}{Z} B^i \right) \\
%
&= \frac{W}{Z+B^2}\left(g^{i\nu}Q_{\nu} - \alpha n^i Q^0\right) + \frac{W}{Z^2+B^2 Z}\left(S_j^{\rm Noble} B^j B^i \right) \\
%
&= \frac{W}{Z+B^2}\left(g^{0i}Q_{0} + g^{ij}Q_{j} + \alpha \frac{\beta^i}{\alpha} u^{\rm Noble}\right) + \frac{W}{Z^2+B^2 Z}\left(S_j^{\rm Noble} B^j B^i \right) \\
%
&= \frac{W}{Z+B^2}\left(\beta^i u^{\rm Noble} + \gamma^{ij}S_{j}^{\rm Noble} + \beta^i u^{\rm Noble}\right) + \frac{W}{Z^2+B^2 Z}\left(S_j^{\rm Noble} B^j B^i \right) \\
%
&= u^{\rm Noble}\frac{2\beta^i W}{Z+B^2} + \frac{S_{j}^{\rm Noble}W}{Z+B^2} \left(\gamma^{ij} + \frac{B^j B^i}{Z} \right) \\
\end{align}
As is hopefully obvious, this equation is significantly more work to compute, especially since $Q^\mu$, $Q_\mu$, and $Q^\mu n_\mu$ are already computed.

After computing $\tilde{u}^i$, we call the `limit_utilde_and_compute_v` function to apply a speed limit to $\tilde{u}^i$ before computing
$$
v^i = \frac{\tilde{u}^i}{u^0} - \beta^i
$$
where $u^0$ is the 0th component of the 4-velocity. If the speed is limited, then $\rho_b$ needs to be recomputed as well
$$
\rho_b = \frac{\rho^{\rm undens}}{\sqrt{-g} u^0} = \frac{\rho^{\rm undens}}{\alpha \psi^6 u^0}
$$

<a id='primitives'></a>

## Step 3.f: Setting the primitives \[Back to [top](#toc)\]
$$\label{primitives}$$

Finally, we update the primitives,

$$
\boxed{
\begin{align}
\rho_{b} &= \rho\\
P &= \left(\Gamma - 1\right)u = P_{\rm cold}\\
v^{i} &= \frac{\tilde{u}^{i}}{u^{0}} - \beta^{i}
\end{align}
}
$$

In [30]:
%%writefile -a $outfile_path__harm_primitives_lowlevel__C


      //The Font fix only sets the velocities.  Here we set the pressure & density HARM primitives.
      if(font_fix_applied==1) {
        prim[RHO] = rho_star_orig/(METRIC_LAP_PSI4[LAPSE]*u0L*METRIC_LAP_PSI4[PSI6]);
        //Next set P = P_cold:
        CCTK_REAL P_cold;

        /**********************************
         * Piecewise Polytropic EOS Patch *
         *  Finding Gamma_ppoly_tab and K_ppoly_tab *
         **********************************/
        /* Here we use our newly implemented
         * find_polytropic_K_and_Gamma() function
         * to determine the relevant polytropic
         * Gamma and K parameters to be used
         * within this function.
         */
        int polytropic_index = find_polytropic_K_and_Gamma_index(eos,prim[RHO]);
        K_ppoly_tab     = eos.K_ppoly_tab[polytropic_index];
        Gamma_ppoly_tab = eos.Gamma_ppoly_tab[polytropic_index];

        // After that, we compute P_cold
        P_cold = K_ppoly_tab*pow(prim[RHO],Gamma_ppoly_tab);

        prim[UU] = P_cold/(Gamma_ppoly_tab-1.0);
      } //Finished setting remaining primitives if there was a Font fix.

      /* Set rho_b */
      PRIMS[RHOB] = prim[RHO];

      /***************
       * PPEOS Patch *
       * Hybrid EOS  *
       ***************
       */
      /* We now compute the pressure as a function
       * of rhob, P_cold, eps_cold, and u = rhob*eps,
       * using the function pressure_rho0_u(), which
       * implements the equation:
       * .-------------------------------------------------------------.
       * | p(rho_b,u) = P_cold + (Gamma_th - 1)*(u - rho_b * eps_cold) |
       * .-------------------------------------------------------------.
       */
      PRIMS[PRESSURE] = pressure_rho0_u(eos, prim[RHO],prim[UU]);

      /* Already set u0L. */
      PRIMS[VX]       = utx_new/u0L - METRIC[SHIFTX];
      PRIMS[VY]       = uty_new/u0L - METRIC[SHIFTY];
      PRIMS[VZ]       = utz_new/u0L - METRIC[SHIFTZ];

      return 0;
    } else {
      //If we didn't find a root, then try again with a different guess.
    }
  }
  CCTK_VInfo(CCTK_THORNSTRING,"Couldn't find root from: %e %e %e %e %e, rhob approx=%e, rho_b_atm=%e, Bx=%e, By=%e, Bz=%e, gij_phys=%e %e %e %e %e %e, alpha=%e",
	     tau_orig,rho_star_orig,mhd_st_x_orig,mhd_st_y_orig,mhd_st_z_orig,rho_star_orig/METRIC_LAP_PSI4[PSI6],rho_b_atm,PRIMS[BX_CENTER],PRIMS[BY_CENTER],PRIMS[BZ_CENTER],METRIC_PHYS[GXX],METRIC_PHYS[GXY],METRIC_PHYS[GXZ],METRIC_PHYS[GYY],METRIC_PHYS[GYZ],METRIC_PHYS[GZZ],METRIC_LAP_PSI4[LAPSE]);
  return 1;
}

//#include "harm_u2p_util.c"
#include "harm_utoprim_2d.c"
#include "eigen.C"
#include "font_fix_hybrid_EOS.C"



Appending to ../src/harm_primitives_lowlevel.C


<a id='font_fix_hybrid_eos'></a>

# Step 4: `font_fix_hybrid_EOS.C` \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos}$$

### Polytropic EOSs

The [Font *et al.*](https://arxiv.org/pdf/gr-qc/9811015.pdf) algorithm (henceforth Font Fix algorithm) can be summarized as follows. Font fixes occur at the atmospheric region, so we start by assuming that $P$ is given only by its cold part, i.e.

$$
P = P_{\rm cold} = K_{\rm atm} \rho_{b}^{\Gamma_{\rm atm}}\ ,
$$

where $K_{\rm atm}$ and $\Gamma_{\rm atm}$ are the constants used in the atmosphere. Then, the specific internal energy is given by

$$
\begin{align}
\epsilon &= \epsilon_{\rm cold}\\
         &= \int d\rho \frac{P_{\rm cold}}{\rho^{2}}\\
         &= K_{\rm atm}\int d\rho \rho^{\Gamma_{\rm atm}-2}\\
         &= \frac{K_{\rm atm}\rho^{\Gamma_{\rm atm}-1}}{\Gamma_{\rm atm}-1}\ .
\end{align}
$$

Having computed $P$ and $\epsilon$, we can compute the enthalpy, $h$, giving

$$
\begin{align}
h &= 1 + \epsilon + \frac{P}{\rho}\\
  &= 1 + \frac{K_{\rm atm}\rho^{\Gamma_{\rm atm}-1}}{\Gamma_{\rm atm}-1} + K_{\rm atm}\rho^{\Gamma_{\rm atm} - 1}\\
  &= 1 + \left(\frac{1}{\Gamma_{\rm atm}-1}+1\right)K_{\rm atm}\rho^{\Gamma_{\rm atm} - 1}\\
\implies &\boxed{ h = 1 + \left(\frac{\Gamma_{\rm atm}}{\Gamma_{\rm atm}-1}\right)K_{\rm atm} \rho^{\Gamma_{\rm atm}-1} }\ .
\end{align}
$$

We then run an iterative process that updates $\rho$ in a consistent way, based on the value of $h$. We now describe this process and its implementation.

<a id='font_fix_hybrid_eos__basic_quantities'></a>

## Step 4.a: Computing the basic quantities needed by the algorithm \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__basic_quantities}$$

We start by computing all basic quantities needed by the Font Fix algorithm:

$$
\boxed{
\begin{align}
\bar{B}^{i}                 &= \frac{B^i}{\sqrt{4\pi}}\\
\bar{B}_{i}                 &= \gamma_{ij}\bar{B}^{j}\\
\bar{B}^{2}                 &= \bar{B}_{i}\bar{B}^{i}\\
\bar{B}                     &= \sqrt{\bar{B}^{2}}\\
\bar{B}\cdot\tilde{S}       &= \bar{B}^{i}\tilde{S}_{i}\\
                    (\bar{B}&\cdot\tilde{S})^{2}\\
\hat{\bar{B}}\cdot\tilde{S} &= \hat{\bar{B}}^{i}\tilde{S}_{i} \equiv \left(\frac{\bar{B}^{i}}{\bar{B}}\right)\tilde{S}_{i}\\
\tilde{S}\cdot\tilde{S} &= \gamma^{ij}\tilde{S}_{i}\tilde{S}_{j}
\end{align}
}\ .
$$

In [31]:
%%writefile $outfile_path__font_fix_hybrid_EOS__C


/**********************************
 * Piecewise Polytropic EOS Patch *
 *    Font fix: function call     *
 **********************************/
inline int font_fix__hybrid_EOS(CCTK_REAL &u_x, CCTK_REAL &u_y, CCTK_REAL &u_z,CCTK_REAL *CONSERVS,CCTK_REAL *PRIMS,CCTK_REAL *METRIC_PHYS,CCTK_REAL *METRIC_LAP_PSI4, eos_struct eos) {

  CCTK_REAL Bxbar = PRIMS[BX_CENTER]*ONE_OVER_SQRT_4PI;
  CCTK_REAL Bybar = PRIMS[BY_CENTER]*ONE_OVER_SQRT_4PI;
  CCTK_REAL Bzbar = PRIMS[BZ_CENTER]*ONE_OVER_SQRT_4PI;
  CCTK_REAL Bbar_x = METRIC_PHYS[GXX]*Bxbar + METRIC_PHYS[GXY]*Bybar + METRIC_PHYS[GXZ]*Bzbar;
  CCTK_REAL Bbar_y = METRIC_PHYS[GXY]*Bxbar + METRIC_PHYS[GYY]*Bybar + METRIC_PHYS[GYZ]*Bzbar;
  CCTK_REAL Bbar_z = METRIC_PHYS[GXZ]*Bxbar + METRIC_PHYS[GYZ]*Bybar + METRIC_PHYS[GZZ]*Bzbar;
  CCTK_REAL B2bar = Bxbar*Bbar_x + Bybar*Bbar_y + Bzbar*Bbar_z;
  CCTK_REAL Bbar = sqrt(B2bar);

  CCTK_REAL check_B_small = fabs(Bxbar)+fabs(Bybar)+fabs(Bzbar);
  if (check_B_small>0 && check_B_small<1.e-150) {
    // need to compute B2bar specially to prevent floating-point underflow
    CCTK_REAL Bmax = fabs(Bxbar);
    if (Bmax < fabs(Bybar)) Bmax=fabs(Bybar);
    if (Bmax < fabs(Bzbar)) Bmax=fabs(Bzbar);
    CCTK_REAL Bxtmp=Bxbar/Bmax, Bytemp=Bybar/Bmax, Bztemp=Bzbar/Bmax;
    CCTK_REAL B_xtemp=Bbar_x/Bmax, B_ytemp=Bbar_y/Bmax, B_ztemp=Bbar_z/Bmax;
    Bbar = sqrt(Bxtmp*B_xtemp + Bytemp*B_ytemp + Bztemp*B_ztemp)*Bmax;
  }
  CCTK_REAL BbardotS = Bxbar*CONSERVS[STILDEX] + Bybar*CONSERVS[STILDEY] + Bzbar*CONSERVS[STILDEZ];
  CCTK_REAL BbardotS2 = BbardotS*BbardotS;
  CCTK_REAL hatBbardotS = BbardotS/Bbar;
  if (Bbar<1.e-300) hatBbardotS = 0.0;
  CCTK_REAL Psim6 = 1.0/METRIC_LAP_PSI4[PSI6];

  // Limit hatBbardotS
  //CCTK_REAL max_gammav = 100.0;
  //CCTK_REAL rhob_max = CONSERVS[RHOSTAR]*Psim6;
  //CCTK_REAL hmax = 1.0 + gam_gamm1_kpoly*pow(rhob_max,gam1);
  //CCTK_REAL abs_hatBbardotS_max = sqrt(SQR(max_gammav)-1.0)*CONSERVS[RHOSTAR]*hmax;
  //if (fabs(hatBbardotS) > abs_hatBbardotS_max) {
  //   CCTK_REAL fac_reduce = abs_hatBbardotS_max/fabs(hatBbardotS);
  //   CCTK_REAL hatBbardotS_max = hatBbardotS*fac_reduce;
  //   CCTK_REAL Bbar_inv = 1.0/Bbar;
  //   CCTK_REAL hat_Bbar_x = Bbar_x*Bbar_inv;
  //   CCTK_REAL hat_Bbar_y = Bbar_y*Bbar_inv;
  //   CCTK_REAL hat_Bbar_z = Bbar_z*Bbar_inv;
  //   CCTK_REAL sub_fact = hatBbardotS_max - hatBbardotS;
  //   CONSERVS[STILDEX] += sub_fact*hat_Bbar_x;
  //   CONSERVS[STILDEY] += sub_fact*hat_Bbar_y;
  //   CONSERVS[STILDEZ] += sub_fact*hat_Bbar_z;
  //   hatBbardotS = hatBbardotS_max;
  //   BbardotS *= fac_reduce;
  //   BbardotS2 = BbardotS*BbardotS;
  //}

  CCTK_REAL sdots = METRIC_PHYS[GUPXX]*SQR(CONSERVS[STILDEX]) + METRIC_PHYS[GUPYY]*SQR(CONSERVS[STILDEY]) + METRIC_PHYS[GUPZZ]*SQR(CONSERVS[STILDEZ])
    + 2.0*( METRIC_PHYS[GUPXY]*CONSERVS[STILDEX]*CONSERVS[STILDEY] + METRIC_PHYS[GUPXZ]*CONSERVS[STILDEX]*CONSERVS[STILDEZ]
            + METRIC_PHYS[GUPYZ]*CONSERVS[STILDEY]*CONSERVS[STILDEZ]);

Writing ../src/font_fix_hybrid_EOS.C


<a id='font_fix_hybrid_eos__sdots'></a>

## Step 4.b: Basic check: looking at $\tilde{S}^{2}$ \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__sdots}$$

We start by looking at the dot product $\tilde{S}^{2}$. Recall that

$$
\tilde{S}_{i} = \left(\rho_{\star}h + \alpha\sqrt{\gamma}\, u^{0}b^{2}\right)u_{i}-\alpha\sqrt{\gamma}\, b^{0}b_{i}\ .
$$

If $\tilde{S}^{2} = 0$, then we must be in a region where $u_{i} = 0 = b_{i}$. In this case, we return

$$
\begin{align}
\rho_{b} &= \psi^{-6}\rho_{\star}\ ,\\
u^{i} &= 0\ ,
\end{align}
$$

and terminate the function call.

In [32]:
%%writefile -a $outfile_path__font_fix_hybrid_EOS__C


  CCTK_REAL rhob;
  if (sdots<1.e-300) {
    rhob = CONSERVS[RHOSTAR]*Psim6;
    u_x=0.0; u_y=0.0; u_z=0.0;
    return 0;
  }
  /* This test has some problem.
     if (fabs(BbardotS2 - sdots*B2bar) > 1e-8) {
     CCTK_VInfo(CCTK_THORNSTRING,"(Bbar dot S)^2, Bbar^2 * sdotS, %e %e",SQR(BbardotS),sdots*B2bar);
     CCTK_VInfo(CCTK_THORNSTRING,"Cauchy-Schwartz inequality is violated!");
     }
  */

Appending to ../src/font_fix_hybrid_EOS.C


<a id='font_fix_hybrid_eos__initial_guesses'></a>

## Step 4.c: Initial guesses for $W$, $S_{{\rm fluid}}^{2}$, and $\rho$ \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__initial_guesses}$$

If $\tilde{S}^{2} \neq 0$, then we move on to the iterative procedure previously mentioned. We start by setting the initial data based on eqs. (A52), (A53), and (A59) found in [Appendix A of Zachariah *et al.* (2012)](https://arxiv.org/pdf/1112.0568.pdf):

$$
\boxed{
\begin{align}
W_{0} &= \psi^{-6}\sqrt{\left(\hat{\bar{B}}\cdot\tilde{S}\right)^{2} + \rho_{\star}^{2}}\\
S_{{\rm fluid},0}^{2} &=\frac{W_{0}^{2}\left(\tilde{S}\cdot\tilde{S}\right)+\left(\bar{B}\cdot\tilde{S}\right)^{2}\left(\bar{B}^{2} + 2W_{0}\right)}{\left(W_{0} + \bar{B}^{2}\right)^{2}}\\
\rho_{0} &= \frac{\psi^{-6}\rho_{\star}}{\sqrt{1+\frac{S_{{\rm fluid},0}^{2}}{\rho_{\star}^{2}}}}
\end{align}
}\ .
$$

In [33]:
%%writefile -a $outfile_path__font_fix_hybrid_EOS__C


  // Initial guess for W, S_fluid and rhob
  CCTK_REAL W0    = sqrt( SQR(hatBbardotS) + SQR(CONSERVS[RHOSTAR]) ) * Psim6;
  CCTK_REAL Sf20  = (SQR(W0)*sdots + BbardotS2*(B2bar + 2.0*W0))/SQR(W0+B2bar);
  CCTK_REAL rhob0 = CONSERVS[RHOSTAR]*Psim6/sqrt(1.0+Sf20/SQR(CONSERVS[RHOSTAR]));

Appending to ../src/font_fix_hybrid_EOS.C


<a id='font_fix_hybrid_eos__main_loop'></a>

## Step 4.d: The main loop \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__main_loop}$$

We now perform the following iterative process, which is again described in [Appendix A of Zachariah *et al.* (2012)](https://arxiv.org/pdf/1112.0568.pdf). We refer the reader to eqs. (A60), (A61), and (A62).

1. Store the previously computed values of $W_{n}$, $S_{{\rm fluid},n}^{2}$, and $\rho_{n}$
2. Compute $h = 1 + \epsilon_{\rm cold} + P_{\rm cold}/\rho_{n}$
3. Set
$$
\boxed{\rho_{n+1} = \psi^{-6}\rho_{\star}\left(1 + \frac{S_{{\rm fluid},n}^{2}}{\rho_{\star} h_{n}}\right)^{-1/2}}
$$

4. For a given value of $n$, perform steps 1 (for $\rho$), 2 and 3 until $\left|\rho_{n+1}-\rho_{n}\right| < \rho_{n+1}\epsilon$, where $\epsilon$ is a user given tolerance
5. After convergence is obtained, update:
$$
\boxed{
\begin{align}
h_{n+1} &= 1 + \epsilon_{\rm cold} + P_{\rm cold}/\rho_{n+1}\\
W_{n+1} &= \psi^{-6}\sqrt{\tilde{S}^{2}_{{\rm fluid},n} + \rho_{\star}^{2} h_{n+1}^{2}}\\
S_{{\rm fluid},n+1}^{2} &= \frac{W^{2}_{n+1}\left(\tilde{S}\cdot\tilde{S}\right) + \left(\bar{B}\cdot\tilde{S}\right)^{2}\left(\bar{B}^{2} + 2W_{n+1}\right)}{\left(W_{n+1} + \bar{B}^{2}\right)^{2}}
\end{align}
}\ .
$$
6. Repeat steps 1 through 5 until $\left|W_{n+1}-W_{n}\right| < W_{n+1}\epsilon$ *and* $\left|S^{2}_{{\rm fluid},n+1}-S^{2}_{{\rm fluid},n}\right| < S^{2}_{{\rm fluid},n+1}\epsilon$ *or* we reach the maximum number of iterations
7. If font fix fails, increase the tolerance and try again.

This is done using the function `font_fix__rhob_loop()`, which is documented in the [inlined functions tutorial notebook](Tutorial-IllinoisGRMHD__inlined_functions.ipynb).

In [34]:
%%writefile -a $outfile_path__font_fix_hybrid_EOS__C


  //****************************************************************
  //                          FONT FIX
  // Impose Font fix when HARM primitives solver fails to find
  //   acceptable set of primitives.
  //****************************************************************

  /* Set the maximum number of iterations */
  int maxits = 500;

  /* Set the allowed tolerance */
  CCTK_REAL tol = 1.e-15;

  /* Declare basic variables */
  int font_fix_status;

    /**********************
   * FONT FIX MAIN LOOP *
   **********************
   * Perform the font fix routine until convergence
   * is obtained and the algorithm returns with no
   * error. Every time the Font fix fails, increase
   * the tolerance by a factor of 10.
   */
  int font_fix_attempts = 5;
  CCTK_REAL font_fix_tol_factor = 10.0;
  for(int n=0; n<font_fix_attempts; n++) {

    tol *= pow(font_fix_tol_factor,n);
    font_fix_status = font_fix__rhob_loop(maxits,tol, W0,Sf20,Psim6,sdots,BbardotS2,B2bar, CONSERVS,eos, rhob0,rhob);
    rhob0 = rhob;
    if(font_fix_status==0) break;

  }

Appending to ../src/font_fix_hybrid_EOS.C


<a id='font_fix_hybrid_eos__outputs'></a>

## Step 4.e: Output $\rho_{b}$ and $u_{i}$ \[Back to [top](#toc)\]
$$\label{font_fix_hybrid_eos__outputs}$$

Finally, we return $\rho_{b}$ and $u_{i}$. In the relations below, $N$ indicates the last computed value of $\rho$ obtained by our iterative process. The quantities evaluated here are

$$
\boxed{
\begin{align}
\rho_{b}   &= \rho_{N}\\
\gamma_{v} &= \frac{\psi^{-6}\rho_{\star}}{\rho_{b}}\\
f_{1}      &= \frac{\psi^{6}\left(\bar{B}\cdot\tilde{S}\right)}{\gamma_{v}\rho_{\star} h}\\
f_{2}      &= \left(\rho_{\star}h + \psi^{6}\frac{\bar{B}^{2}}{\gamma_{v}}\right)^{-1}\\
u_{i}      &= f_{2}\left(\tilde{S}_{i} + f_{1}\bar{B}_{i}\right)
\end{align}
}\ .
$$


In [35]:
%%writefile -a $outfile_path__font_fix_hybrid_EOS__C


  //**************************************************************************************************************

  /* Font fix works! */
  /* First compute P_cold, eps_cold, then h = h_cold */
  CCTK_REAL P_cold, eps_cold;
  compute_P_cold__eps_cold(eos,rhob, P_cold,eps_cold);
  CCTK_REAL h = 1.0 + eps_cold + P_cold/rhob;

  /* Then compute gamma_v using equation (A19) in
   * Etienne et al. (2011) [https://arxiv.org/pdf/1112.0568.pdf]
   * .-----------------------------------------.
   * | gamma_v = psi^{-6} * (rho_star / rho_b) |
   * .-----------------------------------------.
   */
  CCTK_REAL gammav = CONSERVS[RHOSTAR]*Psim6/rhob;

  /* Finally, compute u_{i} */
  CCTK_REAL rhosh = CONSERVS[RHOSTAR]*h;
  CCTK_REAL fac1 = METRIC_LAP_PSI4[PSI6]*BbardotS/(gammav*rhosh);
  CCTK_REAL fac2 = 1.0/(rhosh + METRIC_LAP_PSI4[PSI6]*B2bar/gammav);
  u_x = fac2*(CONSERVS[STILDEX] + fac1*Bbar_x);
  u_y = fac2*(CONSERVS[STILDEY] + fac1*Bbar_y);
  u_z = fac2*(CONSERVS[STILDEZ] + fac1*Bbar_z);

  return 0;
}

Appending to ../src/font_fix_hybrid_EOS.C


<a id='harm_primitives_headers'></a>

# Step 5: `harm_primitives_headers.h` \[Back to [top](#toc)\]
$$\label{harm_primitives_headers}$$

In [36]:
%%writefile $outfile_path__harm_primitives_headers__h
/***********************************************************************************
    Copyright 2006 Charles F. Gammie, Jonathan C. McKinney, Scott C. Noble,
                   Gabor Toth, and Luca Del Zanna

                        HARM  version 1.0   (released May 1, 2006)

    This file is part of HARM.  HARM is a program that solves hyperbolic
    partial differential equations in conservative form using high-resolution
    shock-capturing techniques.  This version of HARM has been configured to
    solve the relativistic magnetohydrodynamic equations of motion on a
    stationary black hole spacetime in Kerr-Schild coordinates to evolve
    an accretion disk model.

    You are morally obligated to cite the following two papers in his/her
    scientific literature that results from use of any part of HARM:

    [1] Gammie, C. F., McKinney, J. C., \& Toth, G.\ 2003,
        Astrophysical Journal, 589, 444.

    [2] Noble, S. C., Gammie, C. F., McKinney, J. C., \& Del Zanna, L. \ 2006,
        Astrophysical Journal, 641, 626.


    Further, we strongly encourage you to obtain the latest version of
    HARM directly from our distribution website:
    http://rainman.astro.uiuc.edu/codelib/


    HARM is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.

    HARM is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with HARM; if not, write to the Free Software
    Foundation, Inc., 51 Franklin St, Fifth Floor, Boston, MA  02110-1301  USA

***********************************************************************************/
#ifndef HARM_PRIMITIVES_HEADERS_H_
#define HARM_PRIMITIVES_HEADERS_H_

static const int NPR =8;
static const int NDIM=4;

/* Adiabatic index used for the state equation */
//#define GAMMA    (2.0)

static const CCTK_REAL G_ISOTHERMAL = 1.0;

/* use K(s)=K(r)=const. (G_ATM = GAMMA) of time or  T = T(r) = const. of time (G_ATM = 1.) */
/*
  #define USE_ISENTROPIC 1

  #if( USE_ISENTROPIC )
  #define G_ATM GAMMA
  #else
  #define G_ATM G_ISOTHERMAL
  #endif
*/

static const int MAX_NEWT_ITER=30;     /* Max. # of Newton-Raphson iterations for find_root_2D(); */
//#define MAX_NEWT_ITER 300     /* Max. # of Newton-Raphson iterations for find_root_2D(); */
static const CCTK_REAL NEWT_TOL    =1.0e-10;    /* Min. of tolerance allowed for Newton-Raphson iterations */
static const CCTK_REAL MIN_NEWT_TOL=1.0e-10;    /* Max. of tolerance allowed for Newton-Raphson iterations */
static const int EXTRA_NEWT_ITER=0; /* ZACH SAYS: Original value = 2. But I don't think this parameter > 0 is warranted. Just slows the code for no reason, since our tolerances are fine. */

static const CCTK_REAL NEWT_TOL2    =1.0e-15;      /* TOL of new 1D^*_{v^2} gnr2 method */
static const CCTK_REAL MIN_NEWT_TOL2=1.0e-10;  /* TOL of new 1D^*_{v^2} gnr2 method */

static const CCTK_REAL W_TOO_BIG    =1.e20;    /* \gamma^2 (\rho_0 + u + p) is assumed
                                                  to always be smaller than this.  This
                                                  is used to detect solver failures */
static const CCTK_REAL UTSQ_TOO_BIG =1.e20;    /* \tilde{u}^2 is assumed to be smaller
                                                  than this.  Used to detect solver
                                                  failures */

static const CCTK_REAL FAIL_VAL     =1.e30;    /* Generic value to which we set variables when a problem arises */

static const CCTK_REAL NUMEPSILON=(2.2204460492503131e-16);


/* some mnemonics */
/* for primitive variables */
static const int RHO    =0;
static const int UU     =1;
static const int UTCON1 =2;
static const int UTCON2 =3;
static const int UTCON3 =4;
static const int BCON1  =5;
static const int BCON2  =6;
static const int BCON3  =7;

/* for conserved variables */
static const int QCOV0  =1;
static const int QCOV1  =2;
static const int QCOV2  =3;
static const int QCOV3  =4;

/********************************************************************************************/
// Function prototype declarations:
int Utoprim_2d(eos_struct eos, CCTK_REAL U[NPR], CCTK_REAL gcov[NDIM][NDIM], CCTK_REAL gcon[NDIM][NDIM],
               CCTK_REAL gdet, CCTK_REAL prim[NPR], long &n_iter);

inline int harm_primitives_gammalaw_lowlevel(const int index,const int i,const int j,const int k,CCTK_REAL *X,CCTK_REAL *Y,CCTK_REAL *Z,
                                             CCTK_REAL *METRIC,CCTK_REAL *METRIC_PHYS,CCTK_REAL *METRIC_LAP_PSI4,
                                             CCTK_REAL *CONSERVS,CCTK_REAL *PRIMS,
                                             CCTK_REAL g4dn[NDIM][NDIM],CCTK_REAL g4up[NDIM][NDIM],
                                             output_stats &stats,eos_struct &eos);

inline int font_fix__hybrid_EOS(CCTK_REAL &u_x, CCTK_REAL &u_y, CCTK_REAL &u_z,CCTK_REAL *CONSERVS,CCTK_REAL *PRIMS,CCTK_REAL *METRIC_PHYS,CCTK_REAL *METRIC_LAP_PSI4, eos_struct eos);
void eigenvalues_3by3_real_sym_matrix(CCTK_REAL & lam1, CCTK_REAL & lam2, CCTK_REAL & lam3,
                                      CCTK_REAL M11, CCTK_REAL M12, CCTK_REAL M13, CCTK_REAL M22, CCTK_REAL M23, CCTK_REAL M33);

/********************************************************************************************/

#endif /* HARM_PRIMITIVES_HEADERS_H_ */



Writing ../src/harm_primitives_headers.h


<a id='code_validation'></a>

# Step 6: Code validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

<a id='driver_conserv_to_prims_validation'></a>

## Step 6.a: `driver_conserv_to_prims.C` \[Back to [top](#toc)\]
$$\label{driver_conserv_to_prims_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [37]:
# Verify if the code generated by this tutorial module
# matches the original IllinoisGRMHD source code

# First download the original IllinoisGRMHD source code
import urllib
from os import path

original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/driver_conserv_to_prims.C"
original_IGM_file_name = "driver_conserv_to_prims-original.C"
original_IGM_file_path = os.path.join(IGM_src_dir_path,original_IGM_file_name)

# Then download the original IllinoisGRMHD source code
# We try it here in a couple of ways in an attempt to keep
# the code more portable
try:
    original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
    # Write down the file the original IllinoisGRMHD source code
    with open(original_IGM_file_path,"w") as file:
        file.write(original_IGM_file_code)
except:
    try:
        original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
        # Write down the file the original IllinoisGRMHD source code
        with open(original_IGM_file_path,"w") as file:
            file.write(original_IGM_file_code)
    except:
        # If all else fails, hope wget does the job
        !wget -O $original_IGM_file_path $original_IGM_file_url

# Perform validation
Validation__driver_conserv_to_prims__C  = !diff $original_IGM_file_path $outfile_path__driver_conserv_to_prims__C

if Validation__driver_conserv_to_prims__C == []:
    # If the validation passes, we do not need to store the original IGM source code file
    !rm $original_IGM_file_path
    print("Validation test for driver_conserv_to_prims.C: PASSED!")
else:
    # If the validation fails, we keep the original IGM source code file
    print("Validation test for driver_conserv_to_prims.C: FAILED!")
    # We also print out the difference between the code generated
    # in this tutorial module and the original IGM source code
    print("Diff:")
    for diff_line in Validation__driver_conserv_to_prims__C:
        print(diff_line)

Validation test for driver_conserv_to_prims.C: FAILED!
Diff:
1c1
< /* We evolve forward in time a set of functions called the 
---
> /* We evolve forward in time a set of functions called the
3,4c3,4
<  *  are updated, we must solve for the primitive variables 
<  *  (rho, pressure, velocities) using a Newton-Raphson 
---
>  *  are updated, we must solve for the primitive variables
>  *  (rho, pressure, velocities) using a Newton-Raphson
6c6
<  *  of the MHD equations again. 
---
>  *  of the MHD equations again.
9,12c9,12
<  *  Raphson solver. Truncation errors in conservative 
<  *  variables can lead to no physical solutions in 
<  *  primitive variables. We correct for these errors here 
<  *  through a number of tricks described in the appendices 
---
>  *  Raphson solver. Truncation errors in conservative
>  *  variables can lead to no physical solutions in
>  *  primitive variables. We correct for these errors here
>  *  through a number of tricks described in the appendices
15,

<a id='harm_primitives_lowlevel_validation'></a>

## Step 6.b: `harm_primitives_lowlevel.C` \[Back to [top](#toc)\]
$$\label{harm_primitives_lowlevel_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [38]:
# Verify if the code generated by this tutorial module
# matches the original IllinoisGRMHD source code

# First download the original IllinoisGRMHD source code
import urllib
from os import path

original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/harm_primitives_lowlevel.C"
original_IGM_file_name = "harm_primitives_lowlevel-original.C"
original_IGM_file_path = os.path.join(IGM_src_dir_path,original_IGM_file_name)

# Then download the original IllinoisGRMHD source code
# We try it here in a couple of ways in an attempt to keep
# the code more portable
try:
    original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
    # Write down the file the original IllinoisGRMHD source code
    with open(original_IGM_file_path,"w") as file:
        file.write(original_IGM_file_code)
except:
    try:
        original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
        # Write down the file the original IllinoisGRMHD source code
        with open(original_IGM_file_path,"w") as file:
            file.write(original_IGM_file_code)
    except:
        # If all else fails, hope wget does the job
        !wget -O $original_IGM_file_path $original_IGM_file_url

# Perform validation
Validation__harm_primitives_lowlevel__C  = !diff $original_IGM_file_path $outfile_path__harm_primitives_lowlevel__C

if Validation__harm_primitives_lowlevel__C == []:
    # If the validation passes, we do not need to store the original IGM source code file
    !rm $original_IGM_file_path
    print("Validation test for harm_primitives_lowlevel.C: PASSED!")
else:
    # If the validation fails, we keep the original IGM source code file
    print("Validation test for harm_primitives_lowlevel.C: FAILED!")
    # We also print out the difference between the code generated
    # in this tutorial module and the original IGM source code
    print("Diff:")
    for diff_line in Validation__harm_primitives_lowlevel__C:
        print(diff_line)

Validation test for harm_primitives_lowlevel.C: FAILED!
Diff:
6c6
< 
---
> #ifndef ENABLE_STANDALONE_IGM_C2P_SOLVER
7a8
> #endif
9,12d9
<   CCTK_REAL kpoly = eos.k_tab[0];
<   CCTK_REAL gamma = eos.gamma_tab[0];
<   int gamma_equals2 = 1;
<   if (fabs(gamma-2.0) > 1.e-10) gamma_equals2 = 0;
15c12
<   CCTK_REAL U[NPR]; 
---
>   CCTK_REAL U[NPR];
42a40
> 
46c44
<   // This is NOT the \mathcal{B}^i, which differs by 
---
>   // This is NOT the \mathcal{B}^i, which differs by
51a50
> 
57a57
> 
60,61c60,61
<     between the two sets of definitions for U and P.  The user may 
<     wish to alter the translation as they see fit.  
---
>     between the two sets of definitions for U and P.  The user may
>     wish to alter the translation as they see fit.
64,72c64,72
<     //         /  rho u^t           \                             //
<     //    U =  |  T^t_t   + rho u^t |  sqrt(-det(g_{\mu\nu}))     //
<     //         |  T^t_i             |                             //
<     //         

<a id='font_fix_gamma_law_validation'></a>

## Step 6.c: `font_fix_gamma_law.C` \[Back to [top](#toc)\]
$$\label{font_fix_gamma_law_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [39]:
# Verify if the code generated by this tutorial module
# matches the original IllinoisGRMHD source code

# First download the original IllinoisGRMHD source code
import urllib
from os import path

original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/font_fix_gamma_law.C"
original_IGM_file_name = "font_fix_gamma_law-original.C"
original_IGM_file_path = os.path.join(IGM_src_dir_path,original_IGM_file_name)

# Then download the original IllinoisGRMHD source code
# We try it here in a couple of ways in an attempt to keep
# the code more portable
try:
    original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
    # Write down the file the original IllinoisGRMHD source code
    with open(original_IGM_file_path,"w") as file:
        file.write(original_IGM_file_code)
except:
    try:
        original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
        # Write down the file the original IllinoisGRMHD source code
        with open(original_IGM_file_path,"w") as file:
            file.write(original_IGM_file_code)
    except:
        # If all else fails, hope wget does the job
        !wget -O $original_IGM_file_path $original_IGM_file_url

# Perform validation
Validation__font_fix_gamma_law__C  = !diff $original_IGM_file_path $outfile_path__font_fix_hybrid_EOS__C

if Validation__font_fix_gamma_law__C == []:
    # If the validation passes, we do not need to store the original IGM source code file
    !rm $original_IGM_file_path
    print("Validation test for font_fix_gamma_law.C: PASSED!")
else:
    # If the validation fails, we keep the original IGM source code file
    print("Validation test for font_fix_gamma_law.C: FAILED!")
    # We also print out the difference between the code generated
    # in this tutorial module and the original IGM source code
    print("Diff:")
    for diff_line in Validation__font_fix_gamma_law__C:
        print(diff_line)

Validation test for font_fix_gamma_law.C: FAILED!
Diff:
1,10d0
< /*-----------------------------------------------------------------------------
<  *
<  * Apply "Font Fix", of Font et al., ONLY when the primitives (con2prim) solver
<  *     fails. Note that this version is OPTIMIZED for gamma=2 polytrope EOS only.
<  *     Application of this fix GUARANTEES (on an analytic level, at least) 
<  *     that con2prim will succeed.
<  *
<  * FIXME: There exist better fixes/strategies now. See McKinney et al's work.
<  *
<  -----------------------------------------------------------------------------*/
12,39d1
< inline int font_fix_gamma_equals2(CCTK_REAL &u_x, CCTK_REAL &u_y, CCTK_REAL &u_z,CCTK_REAL *CONSERVS,CCTK_REAL *PRIMS,CCTK_REAL *METRIC_PHYS,CCTK_REAL *METRIC_LAP_PSI4,eos_struct &eos) {
<   CCTK_REAL rhob;
<   CCTK_REAL kpoly2 = 2.0*eos.K_poly;
<   CCTK_REAL tol = 1.e-15;
<   CCTK_REAL Bxbar = PRIMS[BX_CENTER]*ONE_OVER_SQRT_4PI;
<   CCTK_REAL Bybar = PRIMS[BY_CENTER]*ONE_OVER_SQRT_4

<a id='harm_primitives_headers_validation'></a>

## Step 6.d: `harm_primitives_headers.h` \[Back to [top](#toc)\]
$$\label{harm_primitives_headers_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [40]:
# Verify if the code generated by this tutorial module
# matches the original IllinoisGRMHD source code

# First download the original IllinoisGRMHD source code
import urllib
from os import path

original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/harm_primitives_headers.h"
original_IGM_file_name = "harm_primitives_headers-original.h"
original_IGM_file_path = os.path.join(IGM_src_dir_path,original_IGM_file_name)

# Then download the original IllinoisGRMHD source code
# We try it here in a couple of ways in an attempt to keep
# the code more portable
try:
    original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
    # Write down the file the original IllinoisGRMHD source code
    with open(original_IGM_file_path,"w") as file:
        file.write(original_IGM_file_code)
except:
    try:
        original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
        # Write down the file the original IllinoisGRMHD source code
        with open(original_IGM_file_path,"w") as file:
            file.write(original_IGM_file_code)
    except:
        # If all else fails, hope wget does the job
        !wget -O $original_IGM_file_path $original_IGM_file_url

# Perform validation
Validation__harm_primitives_headers__h  = !diff $original_IGM_file_path $outfile_path__harm_primitives_headers__h

if Validation__harm_primitives_headers__h == []:
    # If the validation passes, we do not need to store the original IGM source code file
    !rm $original_IGM_file_path
    print("Validation test for harm_primitives_headers.h: PASSED!")
else:
    # If the validation fails, we keep the original IGM source code file
    print("Validation test for harm_primitives_headers.h: FAILED!")
    # We also print out the difference between the code generated
    # in this tutorial module and the original IGM source code
    print("Diff:")
    for diff_line in Validation__harm_primitives_headers__h:
        print(diff_line)

Validation test for harm_primitives_headers.h: FAILED!
Diff:
2c2
<     Copyright 2006 Charles F. Gammie, Jonathan C. McKinney, Scott C. Noble, 
---
>     Copyright 2006 Charles F. Gammie, Jonathan C. McKinney, Scott C. Noble,
7c7
<     This file is part of HARM.  HARM is a program that solves hyperbolic 
---
>     This file is part of HARM.  HARM is a program that solves hyperbolic
9,10c9,10
<     shock-capturing techniques.  This version of HARM has been configured to 
<     solve the relativistic magnetohydrodynamic equations of motion on a 
---
>     shock-capturing techniques.  This version of HARM has been configured to
>     solve the relativistic magnetohydrodynamic equations of motion on a
12c12
<     an accretion disk model. 
---
>     an accretion disk model.
14c14
<     You are morally obligated to cite the following two papers in his/her 
---
>     You are morally obligated to cite the following two papers in his/her
17c17
<     [1] Gammie, C. F., McKinney, J. C., \& Toth, 

<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__driver_conserv_to_prims.pdf](Tutorial-IllinoisGRMHD__driver_conserv_to_prims.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [41]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__the_conservative_to_primitive_algorithm.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__the_conservative_to_primitive_algorithm.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__the_conservative_to_primitive_algorithm.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__the_conservative_to_primitive_algorithm.tex
!rm -f Tut*.out Tut*.aux Tut*.log